# ECE/CS 434 | MP2: Wi-Fi Fingerprints
<br />
<nav>
    <span class="alert alert-block alert-warning">Due March 14th 11:59PM 2021 on Gradescope</span> |
    <a href="https://www.gradescope.com/courses/223105">Gradescope</a> | 
    <a href="https://courses.grainger.illinois.edu/cs434/sp2021/">Course Website</a> | 
    <a href="http://piazza.com/illinois/spring2021/csece434">Piazza</a>
</nav><br> 

**Name(s):** _Lukas Dumasius ,NO PARTNER _<br>
**NetID(s):** _lukasd2 ,NO PARTNER _

<hr />  

## Objective
In this MP, you will:
- Implement a localization algorithm based on Wi-Fi fingerprints.
- Experiment with clustering algorithms such as KNN.
- Apply appropriate optimizations to improve localization accuracy.

---
## Imports & Setup
The following `code` cell, when run, imports the libraries you **might** need for this MP. Feel free to delete or import other commonly used libraries. If Gradescope reports an error and you believe it is due to an unsupported import, check with the TA to see if it could be added. 

In [58]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree
from sklearn.linear_model import LinearRegression
from statistics import mean 
import math

---
## Problem Description
There is a room with 3 Wi-Fi access points (their locations are known). You are given the RSSI measurements from all 3 access points (APs) at $N$ unique locations in the room, as well as the phone’s orientation when the measurement was taken. Let’s call this the **“offline phase”**, and call the collected data **"fingerprints"**. In the **“online phase”**, a user in the room walks along a **straight** line. Given the RSSI measurements collected by this user’s smartphone and the Wi-Fi fingerprints collected during the off-line phase, **your task is to find the beginning and end locations of this user’s walk**.

`offline.csv` contains data for the off-line phase and has 5 columns: `(x, y, alpha, SSID, RSSI)`. 

`1-walk-*.csv` contain data collected during the on-line phase and has 3 columns: `(SSID, Time, RSSI)`

`ap_locations_1.csv` contains the coordinates of each AP. This data is only needed if you want to implement optional optimizations. The reference implementation did not use this data.


#### Notes

* Refer to `groundtruth.png` for a visual of the setting.
* `alpha` is provided in degrees.
* `RSSI` is provided in decibels.
* `SSID` provides the unique identifier for each AP.

## Your Implementation
Implement your localization algorithm in the function `wifi_localization(online_file, offline_file, ap_locations)`. Do NOT change its function signature. You are, however, free to define and use helper functions.

In [59]:
# This function takes three arguments: 
#    online_file  (string) - name of data file for online phase
#    offline_file (string) - name of data file for offline phase
#    ap_locations (string) - name of data file for access point locations (optional optimization, reference implementation did not use this)
# It returns a list with two tuple values corresponding to the start and end coordinates of the walk

def wifi_localization(online_file, offline_file, ap_locations):
    
    # This is a data wrangling function to organize the offline files into:
    # local_list_locs - contains tuples of each location tuple((cur_x, cur_y, cur_alpha)
    # local_list_RSSIs - contains tuples of each location tuple((a_RSSI_val, b_RSSI_val, c_RSSI_val)
    # it returns these two lists to the main part of the function
    # args - none
    def location_and_RSSI_collector():
        csvfile_fingerprints = pd.read_csv(offline_file)
        column_x = csvfile_fingerprints["x"]
        column_y = csvfile_fingerprints["y"]
        column_alpha = csvfile_fingerprints["alpha"]
        column_SSID = csvfile_fingerprints["SSID"]
        column_RSSI = csvfile_fingerprints["RSSI"]

        local_list_RSSIs = []
        local_list_locs = []

        cur_x = column_x[0]
        cur_y = column_y[0]
        cur_alpha = column_alpha[0]
        a_RSSI_val = 0
        b_RSSI_val = 0
        c_RSSI_val = 0
        
        for i in np.arange(len(column_x)):                 
        # len(column_x) = len(column_y) = len(column_alpha) = etc....
            if(column_x[i] != cur_x or column_y[i] != cur_y or column_alpha[i] != cur_alpha):  
                # we're entering a new (x,y,alpha)
                # but first we must store our accumulated values for the previous location (x,y,alpha)
                local_list_locs.append(tuple((cur_x, cur_y, cur_alpha)))
                local_list_RSSIs.append(tuple((a_RSSI_val, b_RSSI_val, c_RSSI_val)))
                # and now we move onto moving onto the next entry:
                cur_x = column_x[i]
                cur_y = column_y[i]
                cur_alpha = column_alpha[i]

            # we must do this for all entries (x,y,alpha), new or not               
            if(column_SSID[i] == "a"):
                a_RSSI_val = column_RSSI[i]
            if(column_SSID[i] == "b"):
                b_RSSI_val = column_RSSI[i]
            if(column_SSID[i] == "c"):
                c_RSSI_val = column_RSSI[i]
            
        # special case for the last one (need to append):
        local_list_locs.append(tuple((cur_x, cur_y, cur_alpha)))
        local_list_RSSIs.append(tuple((a_RSSI_val, b_RSSI_val, c_RSSI_val)))
        return (local_list_locs, local_list_RSSIs)
    
##########################################################################    
    # feel free to replace/modify the next two lines
    k_value = 5  # THIS IS HOW MANY NEIGHBORS (In measured/fingerprinted RSSI space) WE AVERAGE ACROSS IN OUR KD TREE
                 # In other words, how many K-NN we "average" location across to return
    
    w_value = 3  # THIS IS THE WINDOW's RADIUS. I.E. how many values from either side of current element  in the online data
                 # to TRY to grab when averaging the RSSI values across a few elements to reduce effect of outliers
                 # this greatly increases the accuracy of the whole algorithm, but you may consider tweaking the values
                 # for different data sets.
                 # You may increase k_value when you feel that theres many outliers in the offline values.
                 # You may increase w_value when you feel that theres many outliers in the online values.
                 # that is the basic jist of when you might want to adjust k and w in this implementation
                 # some edge cases such as first few and last few elements will not try to derefence non-existant elements, so
                 # the window will be a bit smaller there. The linear regression I implemented counteracts most of the effect 
                 # of the few edge cases with a smaller window size, though.
                    
    #### getting online info ####
    
    csvfile_walk = pd.read_csv(online_file)
    column_SSID = csvfile_walk["SSID"]
    column_time = csvfile_walk["Time"]
    column_RSSI = csvfile_walk["RSSI"]
    
    array_column_SSID = np.array(column_SSID)
    array_column_time = np.array(column_time)
    array_column_RSSI = np.array(column_RSSI)
    
    # lets organize the online RSSI a,b,c points into a nice list: list_data_points_online
    cur_SSID = array_column_SSID[0]
    cur_SSID_indx = 0
    list_data_points_online = [[],[],[]]            # 0 = a, 1 = b, 2 = c
    for i in np.arange(array_column_SSID.shape[0]):
        if(array_column_SSID[i] == "a"):
            cur_SSID_indx = 0
        if(array_column_SSID[i] == "b"):
            cur_SSID_indx = 1
        if(array_column_SSID[i] == "c"):
            cur_SSID_indx = 2
        
        list_data_points_online[cur_SSID_indx].append(array_column_RSSI[i])
        
    # list_data_points_online now contains a,b,c RSSI's from time 0 -> n-1 (each in order)
    
     #### done - getting online info ####
    
    list_locs, list_RSSIs = location_and_RSSI_collector()  # list_locs, list_RSSIs corresponding indices are associated. We take advantage of this when querying tree.
    array_locs = np.array(list_locs)                       # this is array of tuples (x,y,alpha), actually the tuples are "array-ized" too
    array_RSSIs = np.array(list_RSSIs)                     # this is array of tuples (a,b,c) corresponding to RSSI vals, actually the tuples are "array-ized" too
    
    # below we leave leaf size, any other things potentially as defaults, shouldnt impact output, just performance
    tree = KDTree(array_RSSIs)
    
    list_windowed_averaged_locations = []                  # these are the final tuples of the walk in form (x, y)
    
    # for each online data point, lets compute an average RSSI, 
    # query tree, and get predicted/averaged/windowed location
    for i in np.arange(len(list_data_points_online[0])):    
        current_avg_RSSI_a = 0
        current_avg_RSSI_b = 0
        current_avg_RSSI_c = 0
        
        current_avg_RSSI_a += list_data_points_online[0][i]
        current_avg_RSSI_b += list_data_points_online[1][i]
        current_avg_RSSI_c += list_data_points_online[2][i]
        for k in np.arange(1, w_value + 1):                        # We need a +1 here because np.arange returns [start,stop)
            if(i+k < len(list_data_points_online[0])):             # (non-inclusive on last element)
                current_avg_RSSI_a += list_data_points_online[0][i+k]
                current_avg_RSSI_b += list_data_points_online[1][i+k]
                current_avg_RSSI_c += list_data_points_online[2][i+k]
            if(i-k >= 0):
                current_avg_RSSI_a += list_data_points_online[0][i-k]
                current_avg_RSSI_b += list_data_points_online[1][i-k]
                current_avg_RSSI_c += list_data_points_online[2][i-k]
        current_avg_RSSI_a /= 2*(w_value)+1
        current_avg_RSSI_b /= 2*(w_value)+1
        current_avg_RSSI_c /= 2*(w_value)+1
        
        
        dist, ind = tree.query([tuple((current_avg_RSSI_a, current_avg_RSSI_b, current_avg_RSSI_c))], k=k_value)
        
        #### averaging over k neighbor's locations ####
        
        list_current_neighbors = []
        current_avg_x = 0
        current_avg_y = 0
   
        for i in np.arange(k_value):
            list_current_neighbors.append(array_locs[ind[0][i]])
        for i in np.arange(k_value):
            current_avg_x += list_current_neighbors[i][0]  # 0 = x
            current_avg_y += list_current_neighbors[i][1]  # 1 = y
    
        current_avg_x /= k_value
        current_avg_y /= k_value
        
        #### done - averaging over k neighbor's locations ####
        
        list_windowed_averaged_locations.append(tuple((current_avg_x, current_avg_y)))

    num_of_walk_elements = len(list_windowed_averaged_locations)
    list_windowed_averaged_locations_just_x = []
    list_windowed_averaged_locations_just_y = []
    for cur_x, cur_y in list_windowed_averaged_locations:
        list_windowed_averaged_locations_just_x.append(cur_x)
        list_windowed_averaged_locations_just_y.append(cur_y)
    
    
    #x_axis_n_samples = np.linspace(0, num_of_walk_elements-1, num_of_walk_elements, dtype=int)
    x_axis_n_samples = np.arange(0, num_of_walk_elements)
    modelx = LinearRegression().fit(np.array(x_axis_n_samples).reshape(-1,1), np.array(list_windowed_averaged_locations_just_x).reshape(-1,1))
    modely = LinearRegression().fit(np.array(x_axis_n_samples).reshape(-1,1), np.array(list_windowed_averaged_locations_just_y).reshape(-1,1))
    
    list_xy_of_interest = [0, num_of_walk_elements-1]        # we are interested in the first (start) and last sample
    array_xy_of_interest = np.array(list_xy_of_interest).reshape(-1,1)
    
    x_final_predicted = modelx.predict(array_xy_of_interest)  # array that contains x_start, x_end
    y_final_predicted = modely.predict(array_xy_of_interest)  # array that contains y_start, y_end
    
 
    return [tuple((x_final_predicted[0][0],y_final_predicted[0][0])), tuple((x_final_predicted[1][0],y_final_predicted[1][0]))] # Your return value should be in this format: [(x_start, y_start), (x_end, y_end)]


---
## Running and Testing
The code below runs and evaluates your localization algorithm on the two datasets provided. We will use this same function `estimate_grade(loc, gt)` to grade your MP on the two provided datasets **and on additional (hidden) datasets**. 

In [60]:
if __name__ == '__main__':
    def estimate_grade(calculated, expected):
        X = 0
        Y = 1
        S = 0
        E = 1
        calculated = np.array(calculated)
        expected = np.array(expected)

        # 1. Calculate deviation of walking direction from ground truth
        def get_deviation(calculated, expected):
            calculated = np.array(calculated[E] - calculated[S])
            expected = np.array(expected[E] - expected[S])
            with np.errstate(divide='ignore', invalid='ignore'): 
                dot_prod = np.dot(calculated, expected) / np.linalg.norm(calculated) / np.linalg.norm(expected)
                deviation = np.nan_to_num(np.degrees(np.arccos(dot_prod)), nan=90)
                return deviation if deviation <= 90 else abs(deviation - 180)

        delta_theta = get_deviation(calculated, expected)

        # You will receive full points if deviation <= 30 degrees. 
        # You will receive 0 points if deviation >= 60 degrees.
        # Points for deviation between 30 and 60 degrees will be scaled proportionally.
        theta_score = 1 if(delta_theta <= 30) else max((1 - abs(delta_theta - 30)/30), 0)

        # 2. Calculating absolute distance between calculated and expected S/E coordinates.
        dist_errors = expected - calculated
        s_dist = np.linalg.norm(dist_errors[S], ord=2)
        e_dist = np.linalg.norm(dist_errors[E], ord=2)

        # You will receive full points if error <= 0.5 units. 
        # You will receive 0 points if error >= 3 units.
        # Points for error between 0.5 and 3 units will be scaled proportionally.
        s_score = 1 if(abs(s_dist) <= 0.5) else max(1 - abs(s_dist - 0.5)/2.5, 0)
        e_score = 1 if(abs(e_dist) <= 0.5) else max(1 - abs(e_dist - 0.5)/2.5, 0)
        dist_score = (s_score + e_score) / 2

        return theta_score * 0.8 + dist_score * 0.2

    student_out_1 = wifi_localization("1-walk-1.csv", "offline.csv", "ap_locations_1.csv")
    expected_1 = [(0., 2.5),(3.5, 2.5)]
    grade_1 = estimate_grade(student_out_1, expected_1)

    student_out_2 = wifi_localization("1-walk-2.csv", "offline.csv", "ap_locations_1.csv")
    expected_2 = [(0., 0.),(3., 3.)]
    grade_2 = estimate_grade(student_out_2, expected_2)

    from IPython.display import display, Markdown
    display(Markdown("""
|  Dataset  |  Expected Output |     Your Output |                  Grade |
|:---------:|-----------------:|----------------:|-----------------------:|
|     1     |     {expected_1} | {student_out_1} |        {grade_1:2.2f}% |
|     2     |     {expected_2} | {student_out_2} |        {grade_2:2.2f}% |
|   Hidden  |              ??? |             ??? | Graded upon Submission | 
|   Hidden  |              ??? |             ??? | Graded upon Submission | 
""".format(
        expected_1=expected_1,
        student_out_1=student_out_1,
        grade_1=(grade_1 * 100),
        expected_2=expected_2,
        student_out_2=student_out_2,
        grade_2=(grade_2 * 100),
    )))


|  Dataset  |  Expected Output |     Your Output |                  Grade |
|:---------:|-----------------:|----------------:|-----------------------:|
|     1     |     [(0.0, 2.5), (3.5, 2.5)] | [(0.8717391304347832, 1.7826086956521734), (2.8500000000000005, 2.391304347826086)] |        96.85% |
|     2     |     [(0.0, 0.0), (3.0, 3.0)] | [(0.8971428571428568, 1.5657142857142856), (2.602857142857143, 2.314285714285714)] |        93.61% |
|   Hidden  |              ??? |             ??? | Graded upon Submission | 
|   Hidden  |              ??? |             ??? | Graded upon Submission | 


---
## Rubric
You will be graded on the two walks provided to you (5 points each) and two additional walks under a different setting (15 points each). Make sure you are not over-fitting to the provided data. We will use the same code from the **Running and Testing** section above to grade all 4 traces of data. In plain English, you will be graded on two error metrics:
- (80%) How much your walking direction deviates from ground truth. You will receive full points if deviation ≤ 30 degrees, and 0 points if deviation ≥ 60 degrees. Points for deviation between 30 and 60 degrees will be scaled proportionally.
- (20%) How much your start and end coordinates deviate from ground truth. You will receive full points if error ≤ 0.5 units. You will receive 0 points if error ≥ 3. Points for error between 0.5 and 3 units will be scaled proportionally.

---
## Submission Guidlines
This Jupyter notebook is the only file you need to submit on Gradescope. If you are working in a pair, make sure your partner is correctly added on Gradescope and that both of your names are filled in at the top of this file.

**Make sure any code you added to this notebook, except for import statements, is either in a function or guarded by `__main__`(which won't be run by the autograder). Gradescope will give you immediate feedback using the provided test cases. It is your responsibility to check the output before the deadline to ensure your submission runs with the autograder.**